# Section 1
according to the lectures, we define the privacy as the action of removing somethign from the db without affecting the statistics of the query.

## Generate parallel databases

In [2]:
import torch

# the number of entries in our database
num_entries = 5000

db = torch.rand(num_entries) > 0.5
db

tensor([0, 1, 1,  ..., 1, 0, 0], dtype=torch.uint8)

In [3]:
db[0:5]

tensor([0, 1, 1, 1, 1], dtype=torch.uint8)

In [4]:
# Create a funcion to remove element x in db.
def get_parallel_db(db, remove_index):
    return torch.cat((db[0:remove_index], db[remove_index+1:]))

In [5]:
get_parallel_db(db, 3)

tensor([0, 1, 1,  ..., 1, 0, 0], dtype=torch.uint8)

In [6]:
# We are now removing x + 1 elemetns of the db and creating a db with at least all
# all the participants removed.

def get_parallel_dbs(db):
    
    parallel_dbs = []
    
    for i in range(len(db)):
        pdb = get_parallel_db(db,i)
        parallel_dbs.append(pdb)
    return parallel_dbs
            

In [7]:
pdbs = get_parallel_dbs(db)


In [8]:
#  Final function thath condeces all the work.

def create_db_and_parallel(num_entries):

    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    return db, pdbs

db, pdbs = create_db_and_parallel(30)
    

In [9]:
db

tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1,
        1, 1, 0, 1, 1, 0], dtype=torch.uint8)

In [10]:
db.shape

torch.Size([30])

In [11]:
len(pdbs)

30

In [12]:
# Section 2

In [15]:
db, pdbs = create_db_and_parallel(30)

In [13]:
def query(db):
    return db.sum()

In [14]:
full_db_result = query(db)
full_db_result

tensor(19)

In [16]:
sensitivity = 0

for pdb in pdbs:
    pdb_result = query(pdb)
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if (db_distance > sensitivity):
        sensitivity = db_distance

In [17]:
sensitivity


tensor(5)

# Sensitivity

The maximun ammount that the query changes  when removing an individual from the datbase is called the **L1 sensitivity** or  **sensitivity**.

In [18]:
def create_db_and_parallel(num_entries):

    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    return db, pdbs

def get_parallel_dbs(db):
    
    parallel_dbs = []
    
    for i in range(len(db)):
        pdb = get_parallel_db(db,i)
        parallel_dbs.append(pdb)
    return parallel_dbs
         

In [42]:
# P3 - Evaluating the privacy of a function

def sensitivity(query, n_entries=1000):
    db, pdbs = create_db_and_parallel(n_entries)
    
    full_db_result = query(db)
    
    max_distance =0
    
    for pdb in pdbs:
        pdb_result = query(pdb)
        db_distance = torch.abs(pdb_result - full_db_result)
        
        if(db_distance > max_distance):
            max_distance = db_distance
    return max_distance

In [43]:
def query(db):
    return db.float().mean()

In [44]:
sensitivity(query)

tensor(0.0005)

In [50]:
 # Project Calculate L1 for threshhold
    
def query(db, threshold = 5):
    return (db.sum() > threshold)

In [64]:
for i in range(10):
    sens_f = sensitivity(query,n_entries=10)
    print(sens_f)

0
0
0
0
0
0
0
0
tensor(255, dtype=torch.uint8)
tensor(255, dtype=torch.uint8)


In [65]:
# Project: Perform a differential Atack

db, _ = create_db_and_parallel(100)


In [67]:
pdb = get_parallel_db(db, remove_index=10)

db[10]

tensor(0, dtype=torch.uint8)

In [68]:
# diff atack using sum query

sum(db) - sum(pdb)

tensor(0, dtype=torch.uint8)

In [76]:
# diff atack using mean query

(sum(db).float() / len(db)) - (sum(pdb).float()/ len(pdb))

tensor(-0.0047)

In [78]:
# Diff atack using threshold
(sum(db).float() > 49) - (sum(pdb).float()  > 49)

tensor(0, dtype=torch.uint8)

# Section 5

Local diff privacy and Global diff privacy

Local diff pravacy??
Each individual adds noice to their data before sending it to the statistical db

Differential privacy always requires a form of randommness or noise added to the query to protect from things like diff atacks

In [ ]:
# Plausible deniability

# Project: Local Diff priv



In [79]:
db, pdbs = create_db_and_parallel(100)



In [88]:
true_result = torch.mean(db.float())
true_result


tensor(0.5400)

In [108]:
first_coin_flip = (torch.rand(len(db)) > 0.5).float()


In [109]:
first_coin_flip

tensor([1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1.,
        0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
        0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1.,
        0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1.,
        1., 1., 1., 0., 1., 0., 1., 1., 1., 0.])